In [1]:
# Imports
load('prismatic_envelope.sage')
load('homology.sage')
load('precision.py')

### User-defined input

In [2]:
# The prime p
p=2

# The motivic weight i in F_p(i)^red
i=2

# The power of the uniformizer n
n=3

# The residual degree f
# The present code only supports totally ramified extensions of Qp,
# i.e., where f=1
f=1

Fprec,total_precision=target_total_precision(p,i,n)

print("total p-adic precision is {}".format(total_precision))
print("Fprec is {}".format(Fprec))

# The coefficient ring W
if f==1:
    W=Zp(p,total_precision,type='capped-abs',print_mode='digits',show_prec=False)
else:
    W=Zq(p**f,total_precision,names='a',type='capped-abs',print_mode='series',show_prec=False)
    
# The Breuil-Kisin ring A
A.<z>=PowerSeriesRing(W,Fprec)

# The Eisenstein polynomial E
# USER DEFINED
E=A(z+p)

total p-adic precision is 20
Fprec is 6


### The main calculation

In [3]:
#%%capture
# Suppresses some Python warnings and SAGE variable injections

# The normalized Eisenstein polynomial
# The normalization is to bring the Eisenstein polynomial into the form E(0)=p
E=eisenstein_normalization(p,E)

if i-p+1>0:
    a_syn0,a_syn1,a_nablaN,a_nablaP,a_gens_N0,a_gens_P0,a_gens_N1,a_gens_P1=syntomic_matrices(p,i-p+1,n,E,total_precision,Fprec,debug=False)
    b_syn0,b_syn1,b_nablaN,b_nablaP,b_gens_N0,b_gens_P0,b_gens_N1,b_gens_P1=syntomic_matrices(p,i,n,E,total_precision,Fprec,debug=False)

ValueError: negative valuation

In [ ]:
class SyntomicComplex:
    def __init__(self,p,i,n,E,total_precision,Fprec,debug=False):
        self.syn0,self.syn1,self.nablaN,self.nablaP,self.gens_0,gens_P0,gens_N1,self.gens_2=syntomic_matrices(p,i,n,E,total_precision,Fprec,debug=False)
        self.gens_1=gens_P0+gens_N1
        self.gens={}
        self.gens[0]=self.gens_0
        self.gens[1]=self.gens_1
        self.gens[2]=self.gens_2
        d0=block_matrix([[self.syn0],[self.nablaN]])
        d1=block_matrix([[self.nablaP,-self.syn1]])
        self.complex=ChainComplex({0:d0,1:d1})
    
    def compute_mod_p_homology(self):
        self.complex_mod_p=ChainComplex({0:Matrix(GF(p),self.complex.differential()[0]),1:Matrix(GF(p),self.complex.differential()[1])})
        self.homology_mod_p=Homology(self.complex_mod_p)
        self.homology_mod_p_orders=self.homology_mod_p.homology_orders
        self.homology_mod_p_dimensions={k:len(v) for k,v in self.homology_mod_p_orders.items()}
        self.homology_mod_p_generators={}
        for k,v in self.homology_mod_p.homology_representatives.items():            
            self.homology_mod_p_generators[k]=[]
            for t in range(v.ncols()):
                gen_name=''
                for s in range(v.nrows()):
                    if v[s,t] != 0:
                        gen_name+='+'+str(v[s,t])+'*'+self.gens[k][s]
                self.homology_mod_p_generators[k].append(gen_name)
            print("The dimension of the mod p cohomology in degree {} is {}.".format(
                k,self.homology_mod_p_dimensions[k]))
            print("The generators of the mnod p cohomology are ")
            print(self.homology_mod_p_generators[k])
            print('\n')

In [ ]:
syn_cplx=SyntomicComplex(p,i-p+1,n,E,total_precision,Fprec,debug=False)

In [ ]:
syn_cplx.compute_mod_p_homology()

In [ ]:
ascii_art(syn_cplx.complex_mod_p)

In [ ]:
a_d0=block_matrix([[a_syn0],[a_nablaN]])
a_d1=block_matrix([[a_nablaP,-a_syn1]])
a_d0_Fp=Matrix(GF(p),a_d0)
a_d1_Fp=Matrix(GF(p),a_d1)
a_C=ChainComplex({0:a_d0_Fp,1:a_d1_Fp})
a_C.homology()

In [ ]:
b_d0=block_matrix([[b_syn0],[b_nablaN]])
b_d1=block_matrix([[b_nablaP,-b_syn1]])
b_d0_Fp=Matrix(GF(p),b_d0)
b_d1_Fp=Matrix(GF(p),b_d1)
b_C=ChainComplex({0:b_d0_Fp,1:b_d1_Fp})
b_C.homology()

In [ ]:
ascii_art(a_C)

In [ ]:
ascii_art(b_C)

# Computing v1

In [ ]:
v1N0,v1P0,v1N1,v1P1=v1_matrices(p,i,n,E,total_precision,Fprec,debug=False)

In [ ]:
v1N0_Fp=Matrix(GF(p),v1N0)
v1P0_Fp=Matrix(GF(p),v1P0)
v1N1_Fp=Matrix(GF(p),v1N1)
v1P1_Fp=Matrix(GF(p),v1P1)

In [ ]:
v1N0_Fp

In [ ]:
v1P0_Fp

In [ ]:
v1N1_Fp

In [ ]:
v1P1_Fp

In [ ]:
v1_0=Matrix(GF(p),v1N0)
v1_1=Matrix(GF(p),block_matrix([[v1P0,0],[0,v1N1]]))
v1_2=Matrix(GF(p),v1P1)

In [ ]:
v1_0

In [ ]:
v1_1

In [ ]:
v1_2

In [ ]:
v1_1*a_C.differential()[0]

In [ ]:
b_C.differential()[0]*v1_0

In [ ]:
v1_2*a_C.differential()[1]==b_C.differential()[1]*v1_1

In [ ]:
v1={0:v1_0,1:v1_1,2:v1_2}
ha_C = Homology(a_C)
hb_C = Homology(b_C)
mor = MorphismHomology(ha_C, hb_C, v1)

for h in mor.hF.values():
    print(ascii_art(h))
    print('\n')

In [ ]:
ascii_art(ha_C.complex_smith)

In [ ]:
ascii_art(hb_C.complex_smith)

In [ ]:
ha_C.homology_representatives[1]

In [ ]:
ha_C.homology_orders[1]

In [ ]:
red_0=block_matrix([[-a_C.differential()[0]],[-v1[0]]])
red_1=block_matrix([[-a_C.differential()[1],0],[-v1[1],b_C.differential()[0]]])
red_2=block_matrix([[-v1[2],b_C.differential()[1]]])
reduced_complex=ChainComplex({-1:red_0,0:red_1,1:red_2})
ascii_art(reduced_complex)

In [ ]:
reduced_homology=Homology(reduced_complex)

In [ ]:
reduced_homology.homology_orders

In [ ]:
M=Matrix(ZZ,[[1,2,3],[4,5,6],[7,8,9]]);M

In [ ]:
len(M.columns()[0])

In [ ]:
M.columns()[0][2]